In [ ]:
# import necessary packages
import requests
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import json

/Users/longer/nhl/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
# initialize api key to .txt file
api_key = 'soccer_api.txt'

In [ ]:
# setup api according to documentation
headers = {
    'x-rapidapi-host': 'v3.football.api-sports.io',
    'x-rapidapi-key' : api_key
}

# setup function to extract data from the api
def get_api_key(endpoint):
    url = f'https://v3.football.api-sports.io/{endpoint}'
    response = requests.get(url, headers=headers)
    return response.json()


In [ ]:
# example of api call to retrieve leagues
leagues = get_api_key('/leagues')
with open ('leagues.json', 'w') as f:
    json.dump(leagues, f, indent=2)